In [4]:
workspace() #hierdurch werden Warnungen und Fehlermeldungen behoben. wieso genau weiß ich nicht.

type state #der Typ state soll einen Zustand beschreiben
    j::Int64 #(Binär-)Zahl des Zustands
    n::Int64 #Anzahl der Gitterplätze
    f::Int64 #Faktor. Ist generell ersteinmal 1. Wird eventuell zu -1 verändert, wenn ein Operator auf den Zustand wirkt.
end

function getStateArray(zustand::state) #diese Funktion empfängt einen Zustand vom Typ state. Für diesen Zustand ermittelt 
    a=zeros(Int64, 2*zustand.n) #sie das zugehörige Array mit den Besetzungszahlen
    number = zustand.j
    counter=1
    while number > 0
        a[counter]=number%2
        number= number%2 == 1 ? Int64(number/2 - 0.5) : number/2
        counter=counter+1
    end
    return a
end


#Die Funktion CreateSpin wirkt auf ein Objekt vom Typ state so wie der Erzeugeroperator auf einen Zustand.
function CreateSpin(i,s,zustand) #i ist der Gitterplatz, s ist der Spin, mit Wert 2 für up und 1 für down
   if isa(zustand,Int64) true
        b=0
    else
   number=zustand.j
   occupationnumber= getStateArray(zustand)[2i-s+1]
   newnumber = number + 2^(2*i-s)
    a=0
    for j in 1:2i-s
            a+= getStateArray(zustand)[j]
    end
   newfactor = (1-occupationnumber)*zustand.f*(-1)^a
   b= newfactor == 0 ? 0:  newstate= state(newnumber,zustand.n,newfactor)
    end
return b
end

#Die Funktion AnnihilateSpin wirkt auf ein Objekt vom Typ state so wie der Vernichteroperator auf einen Zustand.
function AnnihilateSpin(i,s,zustand) #i ist der Gitterplatz, s ist der Spin, mit Wert 2 für up und 1 für down
    if isa(zustand,Int64) true
      b=0
    else
     number=zustand.j
     occupationnumber= getStateArray(zustand)[2i-s+1] 
     a=0
      for j in 1:2i-s
            a+= getStateArray(zustand)[j]
      end
     newfactor = occupationnumber*zustand.f*(-1)^a
     newnumber = number - 2^(2*i-s)
     b= newfactor == 0 ? 0:  newstate= state(newnumber,zustand.n,newfactor)
    end
   return b
end

#Wichtig: Die Operatoren wirken, egal in welcher Zusammensetzung untereinander, nur so, dass ein Zustand entweder Null wird oder zu einem anderen Zustand vom Typ state. Wichtig ist, dass zu keinem Zeitpunkt eine Linearkombination aus zwei Zuständen entsteht. Ansonsten funktioniert mein Programm so nicht mehr.
#Hier ist die Funktion um eine Hamiltonmatrix zu erstellen, für n Gitterplätze mit Interaktion
function Ham(E,U,n) #E ist der Energieterm. U ist das "Hubbard U".
    H=zeros(4^n,4^n) #leere Hamiltonmatrix der entsprechenden Dimension
for j in 0:4^n-1
        y=state(j,n,1) #y sind die Zustände, auf die der Hamiltonoperator vorerst wirkt
    for i in 1:n 
        for s in 1:2  #erster Summenterm mit E
            x=CreateSpin(i,s,AnnihilateSpin(i,s,y)) #x sind die Zustände, die entstehen, nachdem der erste Teil des Hamiltonoperators auf die Zustände y angewendet wurde
            if x==0 
                for k in 1:4^n #wenn der gesamte Zustand 0 wird, so ist gesamte i-te Spalte 0. Im Prinizp ist dieser Schritt nicht nötig, da H bereits eine Nullmatrix ist. 
                H[i,k]=+ 0
                end
            elseif isa(x,state) true
                H[(x.j)+1,(y.j)+1]+= E*x.f#Für jeden neu erzeugten Zustand x findet sich ein Matrixelement H[x,y]mit entsprechendem Element E und Faktor x.f, welche 1 oder -1 sein kann.
            end
        end
            #zweiter Summenterm mit U
        x=CreateSpin(i,2,AnnihilateSpin(i,2,CreateSpin(i,1,AnnihilateSpin(i,1,y))))  
            if x==0
                for k in 1:4^n
                H[i,k]+= 0
                end
            elseif isa(x,state) true
                H[(x.j)+1,(y.j)+1]+= U*x.f 
            end        
      end
            #dritter Summenterm mit t und nächsten Nachbarn
      for s in 1:2
           x=CreateSpin(i,s,AnnihilateSpin(i,s,y)) 
      end
        
end
return H
end
#Ham(1,1,1) #wie wird meine Matrix vollständig ausgegeben? (ohne "..."). bei n=8 erscheint OutOfMemoryError(). Wie kann ich das Programm verändern, sodass weniger Speicher verbraucht wird?getStateArray(state(0,2,1))

function tightbinding(t,n)
    B=zeros(4^n,4^n) #leere Hamiltonmatrix der entsprechenden Dimension
    for j in 0:4^n-1
        y=state(j,n,1)
         for s in 1:2
            i=1
            while i+1<=n
                x=CreateSpin(i,s,AnnihilateSpin(i+1,s,y))
                 if isa(x,state) true
                 B[(x.j)+1,(y.j)+1]+= t*x.f 
                 end
              i+=1  
            end
                    k=1
            while k+1<=n
                x=CreateSpin(k+1,s,AnnihilateSpin(k,s,y))
                 if isa(x,state) true
                 B[(x.j)+1,(y.j)+1]+= t*x.f#Für jeden neu erzeugten Zustand x findet sich ein Matrixelement H[x,y]mit entsprechendem Element E und Faktor x.f, welche 1 oder -1 sein kann.
                 end
              k+=1  
            end
            end
      
        
    end
return B
end


LoadError: MethodError: no method matching Ham(::Int64, ::Int64)[0m
Closest candidates are:
  Ham(::Any, ::Any, [1m[31m::Any[0m) at In[4]:61[0m